### Блок тестовых вопросов

*В каком случае ROC-AUC может не быть хорошим показателем качества?*: 

1. классификатор случайным образом ранжирует объекты
2. имеет место существенный дисбаланс в сторону негативного класса, при этом есть информация о том, что абсолютное большинство объектов негативного класса правильно ранжируется большинством алгоритмов (так называемые easy negatives)
3. выходы классификатора не являются оценками вероятностей	
4. классификатор переобучен


**Ответ: 2)** пример easy negatives на слайде из лекции

<div>
<img src="easy_negatives.png" width="1000"/>
</div>
_______________________________________________________

*Какой нужно взять порог в линейном классификаторе, чтобы посчитать AUC-ROC?*: 

1.	Максимальный
2.	Минимальный
3.  Тот, при котором точность и полнота максимальны
4.	Никакой, AUC-ROC считается сразу для всех значений порогов


**Ответ: 4)** по построению и определению это так

___________________________________________

*ROC-AUC равен 0. Какие выводы можно сделать?*: 

1.	Модель ужасная, нужно заново обучить
2.	Модель отличная, стоит лишь перевернуть получившийся список отсортированных объектов
3.	Нельзя ничего сказать в этом случае
4.	Модель выдает случайные числа в качестве меры уверенности на каждом объекте


**Ответ: 2)** Нулевой $ROC-AUC$ означает, что если мы для всех предсказаний отсортируем $P(Y=+1|X)$ в порядке возрастания, то в начале окажутся отрицательные объекты, а потом положительные. Тогда предсказания можно просто развернуть, $P^{new}(Y=+1|X) = 1 -  P(Y=+1|X)$ и модель окажется идеальной с $ROC-AUC=1$

___________________________________________

### Блок практики

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Загрузим датасет

data = pd.read_csv('processed_train.csv')
data.head()

In [ ]:
### Убедимся в дисбалансе классов

data.HasDetections.value_counts()

Разделим выборку на тренировочную и тестовую

In [ ]:
y = data['HasDetections']
X = data.drop('HasDetections', axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(X, y, 
                                                     test_size=0.25, 
                                                     shuffle=True, 
                                                     random_state=1)

Обучите логистическую регрессию на тренировочной части наших данных, используя класс `LogisticRegression` из `sklearn.linear_model` с `l2` регуляризацией и коэффициентом регуляризации `C=0.01`. Ваш estimator должен нормировать данные по статистикам, рассчитывающимся на тренировочной выборке. Придется использовать конструкцию `Pipeline`. Положите все в переменную `pipe`.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

### Построим pipeline с масштабированием
### Учитывая параметры из условия

pipe = Pipeline([('scaler', StandardScaler()), 
                 ('LR', LogisticRegression(penalty='l2', 
                                           C=0.01))])

pipe.fit(X_train, y_train)

Постройте ROC-кривую с помощью `RocCurveDisplay` и посчитайте `ROC-AUC` любым из показанных на практике способов. Один из вариантов - построить кривую с помощью `roc_curve` из `sklearn.metrics` и массивы пар `tpr-fpr` передать в `auc` из того же модуля `sklearn`. Проделайте эту операцию как для предсказаний на трейне, так и на тесте!

P.S. `roc_curve` необходимо скормить истинные метки классов (y_train/y_test), а также предсказанные вероятности (воспользуйтесь методом `predict_proba`)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay

### roc_curve вернет тюпл из наборов (FPR, TPR)
### для всевозможных порогов _

fpr, tpr, _ = roc_curve(y_train,pipe.predict_proba(X_train)[:, 1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

In [ ]:
### Теперь на тесте

fpr_test, tpr_test, __test = roc_curve(y_test,pipe.predict_proba(X_test)[:, 1])
roc_display = RocCurveDisplay(fpr=fpr_test, tpr=tpr_test).plot()

In [ ]:
from sklearn.metrics import auc 

### Замерим ауки, передав в auc все наборы (FPR, TPR)
auc(fpr, tpr)

In [ ]:
### На тесте

auc(fpr_test, tpr_test)

Постройте PR-кривые на трейне и на тесте. Посчитайте PR-AUC

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import precision_recall_curve

### Аналогичный синтаксис

precision, recall, thresholds = precision_recall_curve(y_train, pipe.predict_proba(X_train)[:, 1])

display = PrecisionRecallDisplay(precision=precision, recall=recall).plot()

In [ ]:
auc(recall, precision)

In [ ]:
precision_test, recall_test, thresholds_test = precision_recall_curve(y_test,
                                                                      pipe.predict_proba(X_test)[:, 1])

display = PrecisionRecallDisplay(precision=precision_test, recall=recall_test).plot()

In [ ]:
auc(recall_test, precision_test)

Построим калибровочную кривую. Хорошо ли откалиброван обученный классификатор? Сравним с калибровочной кривой для случайного классификатора!

In [ ]:
from sklearn.calibration import calibration_curve
plt.figure(figsize=(8, 5))
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

### Предсказания модели на тесте и соответствующая калибровочная кривая
prob_pos = pipe.predict_proba(X_test)[:, 1]
fraction_of_positives, mean_predicted_value = calibration_curve(y_test, prob_pos, n_bins=10)
plt.plot(mean_predicted_value, fraction_of_positives, "s-", label="logreg")

### Пусть наши предсказания - случайные, то есть для каждого объекта
### вероятность принадлежности + классу это просто случайное число [0; 1]
prob_pos = np.random.uniform(size=(X_test.shape[0],))
fraction_of_positives, mean_predicted_value = calibration_curve(y_test, prob_pos, n_bins=10)
plt.plot(mean_predicted_value, fraction_of_positives, "s-", label="random")

plt.ylabel("Fraction of positives")
plt.xlabel("Mean predicted value")
plt.ylim([-0.05, 1.05])
plt.legend(loc="lower right")
plt.title('Calibration plots  (reliability curve)')
plt.tight_layout()

--*Классификатор достаточно хорошо откалиброван - кривая близка к идеальной (диагональной). Но того и стоило ожидать, из теоретических свойсв ЛогРега как раз-таки есть корректная оценка вероятностей!*